In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img, img_to_array # Import load_img and img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from google.colab import files

# Define the dataset path
dataset_path ='/content/drive/MyDrive/person_images'

# Create the image data generator
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% validation split
)

# Create the training generator
train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# Create the validation generator
val_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)
# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    # Add the missing closing bracket
])
layers = [
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
]

Found 887 images belonging to 4 classes.
Found 221 images belonging to 4 classes.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 111, 111, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 26, 26, 128)      

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=25
)


Epoch 1/25
27/27 [==============================] - 241s 9s/step - loss: 0.6433 - accuracy: 0.7673 - val_loss: 0.1101 - val_accuracy: 0.9271
Epoch 2/25
27/27 [==============================] - 138s 5s/step - loss: 0.0257 - accuracy: 0.9942 - val_loss: 0.0060 - val_accuracy: 1.0000
Epoch 3/25
27/27 [==============================] - 149s 6s/step - loss: 0.1851 - accuracy: 0.9357 - val_loss: 0.1860 - val_accuracy: 0.9062
Epoch 4/25
27/27 [==============================] - 138s 5s/step - loss: 0.0669 - accuracy: 0.9871 - val_loss: 0.0057 - val_accuracy: 1.0000
Epoch 5/25
27/27 [==============================] - 148s 5s/step - loss: 0.0081 - accuracy: 0.9977 - val_loss: 0.0057 - val_accuracy: 1.0000
Epoch 6/25
27/27 [==============================] - 136s 5s/step - loss: 0.0258 - accuracy: 0.9965 - val_loss: 0.0244 - val_accuracy: 0.9948
Epoch 7/25
27/27 [==============================] - 153s 6s/step - loss: 0.0066 - accuracy: 0.9988 - val_loss: 0.0062 - val_accuracy: 1.0000
Epoch 8/25
27

In [ ]:
# Evaluate the model on the validation data
val_loss, val_acc = model.evaluate(val_generator, steps=val_generator.samples // val_generator.batch_size)
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_acc}')


4/4 [==============================] - 6s 1s/step - loss: 6.5193e-09 - accuracy: 1.0000
Validation Loss: 6.51925713412993e-09
Validation Accuracy: 1.0


In [ ]:
# Save the model to Google Drive
model_save_path = '/content/drive/MyDrive/trainedmodel/identifier.h5'  # Change this path as needed
model.save(model_save_path)
print(f'Model saved to {model_save_path}')


In [ ]:
from google.colab import files
import numpy as np
from tensorflow.keras.preprocessing import image

# Function to preprocess the uploaded image
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Rescale image
    return img_array

# Upload an image
uploaded = files.upload()

for img_name in uploaded.keys():
    img_path = f'/content/{img_name}'
    img_array = preprocess_image(img_path)

    # Predict the class
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions[0])
    class_labels = list(train_generator.class_indices.keys())
    predicted_label = class_labels[predicted_class]

    print(f'Predicted Label: {predicted_label}')

    # Display the image
    from IPython.display import display, Image
    display(Image(img_path))
